In [103]:
import numpy as np
import pandas as pd

from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests 

# Task 2

In [104]:
AUCd = pd.read_csv('AUCs.txt', sep='\t')

In [105]:
AUCd.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [106]:
AUCd.columns

Index([u'Unnamed: 0', u'C4.5', u'C4.5+m', u'C4.5+cf', u'C4.5+m+cf'], dtype='object')

In [107]:
cols = [u'C4.5', u'C4.5+m', u'C4.5+cf', u'C4.5+m+cf']

In [108]:
p_data = []

for i, column1 in enumerate(AUCd[cols]):
    for j, column2 in enumerate(AUCd[cols]):
        if i>=j:
            continue
            
        p = stats.wilcoxon(AUCd[column1]-AUCd[column2])[1]
        p_data.append([column1, column2, p])

In [109]:
p_data = pd.DataFrame.from_records(p_data)
p_data.columns = ['col1', 'col2', 'p']
p_data

,col1,col2,p
0,C4.5,C4.5+m,0.010757
1,C4.5,C4.5+cf,0.861262
2,C4.5,C4.5+m+cf,0.015906
3,C4.5+m,C4.5+cf,0.046333
4,C4.5+m,C4.5+m+cf,0.327826
5,C4.5+cf,C4.5+m+cf,0.022909


# Task 3

In [110]:
(p_data.p<0.05).value_counts()

True     4
False    2
Name: p, dtype: int64

# Task 5

In [111]:
reject, p_corrected, a1, a2 = multipletests(p_data.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [112]:
p_data_holm = p_data
p_data_holm['p_corrected'] = p_corrected 
p_data_holm['reject'] = reject
p_data_holm

,col1,col2,p,p_corrected,reject
0,C4.5,C4.5+m,0.010757,0.064543,False
1,C4.5,C4.5+cf,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,0.022909,0.091636,False


In [114]:
(p_data_holm.p_corrected<0.05).value_counts()

False    6
Name: p_corrected, dtype: int64

# Task 6

In [115]:
reject, p_corrected, a1, a2 = multipletests(p_data.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [116]:
p_data_bh = p_data
p_data_bh['p_corrected'] = p_corrected 
p_data_bh['reject'] = reject
p_data_bh

,col1,col2,p,p_corrected,reject
0,C4.5,C4.5+m,0.010757,0.045818,True
1,C4.5,C4.5+cf,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,0.022909,0.045818,True


In [117]:
(p_data_bh.p_corrected<0.05).value_counts()

True     3
False    3
Name: p_corrected, dtype: int64